In [1]:
import uproot
import glob
import sqlite3
import numpy as np
from tqdm import tqdm

In [2]:
# filelist = "/exp/sbnd/app/users/munjung/xsec/wienersvd/cafpyana/data/sample_lists/sbnd/prod_2025B/MCP2025B_fullpath.list"
filelist = "/exp/sbnd/app/users/munjung/misc/filelists/data/MCP2025B_DevData.list"

with open(filelist, 'r') as f:
    files = [line.strip() for line in f if line.strip()]

print("number of files: ", len(files))
print("first 3 files: ", files[:3])

number of files:  2924
first 3 files:  ['/pnfs/sbn/data_add/sbn_nd/poms_production/data/MCP2025B_02/v10_06_00_02/DevSample/flatcaf/bnblight/e4/reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder7_art1_run18255_23_strmBNBLight_20250217T101850-e4673f61-d0de-9ed9-12d8-d45db898029c.flat.caf.root', '/pnfs/sbn/data_add/sbn_nd/poms_production/data/MCP2025B_02/v10_06_00_02/DevSample/flatcaf/bnblight/be/reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder2_art2_run18255_54_strmBNBLight_20250218T052009-be9634d2-18b0-eb7c-0e87-57a1d3c5540d.flat.caf.root', '/pnfs/sbn/data_add/sbn_nd/poms_production/data/MCP2025B_02/v10_06_00_02/DevSample/flatcaf/bnblight/ad/reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder2_p2_art2_run18255_31_strmBNBLight_20250217T152038-adee79da-9586-1ca2-f5fa-c5d522df80ae.flat.caf.root']


# Make db

In [3]:
conn = sqlite3.connect("/exp/sbnd/data/users/munjung/xsec/2025B/Data_dev-meta_lookup.db")
cur = conn.cursor()

In [4]:
cur.execute("DROP TABLE IF EXISTS lookup")
cur.execute("""
    CREATE TABLE lookup (
        run INTEGER,
        subrun INTEGER,
        event INTEGER,
        file TEXT
    )
""")

In [ ]:
for i in range(0, len(files), 2000)[:-1]:
    print(f"Processing range {i} to {i + 2000}")
    startn = i
    endn = i + 2000

    runs = []
    subruns = []
    events = []
    fnames = []

    import concurrent.futures

    def process_file(fname):
        with uproot.open(fname) as caf:
            tree = caf["recTree"]
            this_runs = tree["rec.hdr.run"].array(library="np")
            this_subruns = tree["rec.hdr.subrun"].array(library="np")
            this_events = tree["rec.hdr.evt"].array(library="np")
            return (
                list(this_runs),
                list(this_subruns),
                list(this_events),
                [fname]*len(this_runs)
            )

    results = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=30) as executor:
        for result in tqdm(executor.map(process_file, files[startn:endn]), total=len(files[startn:endn])):
            results.append(result)

    for this_runs, this_subruns, this_events, these_fnames in results:
        runs.extend(this_runs)
        subruns.extend(this_subruns)
        events.extend(this_events)
        fnames.extend(these_fnames)

    cur.executemany(
        "INSERT INTO lookup (run, subrun, event, file) VALUES (?, ?, ?, ?)",
        zip(runs, subruns, events, fnames)
    )

100%|██████████| 924/924 [00:43<00:00, 21.05it/s]


In [7]:
conn.commit()
conn.close()

In [129]:
# runs = np.array(runs)
# subruns = np.array(subruns)
# events = np.array(events)
# fnames = np.array(fnames)
# fnames

# Query for filename

In [3]:
import pandas as pd

In [4]:
def find_files(cur, run, subrun, event):
    def to_bytes(x):
        if isinstance(x, int):
            return x.to_bytes(4, byteorder='little')
        return x
    run = to_bytes(run)
    subrun = to_bytes(subrun)
    event = to_bytes(event)
    cur.execute(
        "SELECT file FROM lookup WHERE run=? AND subrun=? AND event=?",
        (run, subrun, event)
    )
    return [row[0] for row in cur.fetchall()]

In [5]:
# conn = sqlite3.connect("/exp/sbnd/data/users/munjung/xsec/2025B/MC_bnb_cosmics-meta_lookup.db")
conn = sqlite3.connect("/exp/sbnd/data/users/munjung/xsec/2025B/Data_dev-meta_lookup.db")
cur = conn.cursor()

In [6]:
# cur.execute("SELECT DISTINCT run, subrun, event FROM lookup")
# unique_meta = cur.fetchall()
# print("found {} unique meta entries".format(len(unique_meta)))
# for r, s, ev in unique_meta[:10]:
#     print(r, s, ev)

In [7]:
!ls event_list

all.csv		 numu_cc_np0pi_meta.csv  nu_other_meta.csv
cosmic_meta.csv  numu_cc_other_meta.csv  nu_outfv_meta.csv
data_meta.csv	 numu_nc_meta.csv	 signal_meta.csv


In [6]:
# find_files(cur, 3998, 67, 14)[0].split("/")[-1]

In [7]:
# meta_csv = "event_list/numu_nc_meta.csv"
meta_csv = "event_list/data_meta.csv"
df = pd.read_csv(meta_csv)

for i in range(10):
    this_run = int(df.loc[i].run)
    this_subrun = int(df.loc[i].subrun)
    this_evt = int(df.loc[i].evt)
    print(this_run, this_subrun, this_evt)
    print(find_files(cur, this_run, this_subrun, this_evt)[0].split("/")[-1])

18255 1 143434
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder7_art1_run18255_18_strmBNBLight_20250217T070400-36039bb5-f26c-7f30-4f10-5373fbd32bac.flat.caf.root
18345 1 468552
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder3_p2_art2_run18345_58_strmBNBLight_20250324T045208-bd443937-99e9-3423-a6f3-36d22e7b6621.flat.caf.root
18345 1 468552
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder3_p2_art2_run18345_58_strmBNBLight_20250324T045208-bd443937-99e9-3423-a6f3-36d22e7b6621.flat.caf.root
18255 1 500859
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder3_p2_art2_run18255_64_strmBNBLight_20250218T115554-5a58f92b-59bf-cc5d-8ef9-fa9f85235e34.flat.caf.root
18268 1 77337
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder3_p2_art2_run18268_10_strmBNBLight_20250224T212651-f7d7e148-ae4f-4615-82c8-484e9429ee27.flat.caf.root
18268 1 77337
reco2_reco1_filtered_decoded-raw_filtered_data_EventBuilder3_p2_art2_run18268_10_strmBNBLight_20250224T212651-